In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
import pandas as pd
import time
import requests
from tqdm import tqdm
import random
from selenium.common.exceptions import NoSuchElementException

In [21]:
def WildFlower():
    wait = WebDriverWait(driver, 10)
    WildFlower = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[3]/nav/ul/li[1]/a")))  
    WildFlower.click()
    time.sleep(5)
    driver.implicitly_wait(10)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')   
    div = soup.find_all('div', id='wildflowers_subcategories')
    subCategory=[]
    for i in div:
            anchor_tags = i.find_all('a')
            for anchor_tag in anchor_tags:
                href_value = anchor_tag.get('href')
                subCategory.append(href_value)
    return subCategory

In [22]:
def FlowerSeed():
    wait = WebDriverWait(driver, 10)
    FlowerSeed = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[3]/nav/ul/li[2]/a")))  
    FlowerSeed.click()
    time.sleep(5)
    driver.implicitly_wait(10)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    div = soup.find_all('div', class_='text_root__bzY2i pb-text-root')
    subCategory=[]
    for i in div:
        anchor_tags = i.find_all('a')
        for anchor_tag in anchor_tags:
            href_value = anchor_tag.get('href')
            if href_value and 'https://www.americanmeadows.com/category' in href_value:
                subCategory.append(href_value)     
    return subCategory

In [11]:
def Get_ProductLinks(numeric_part):
    URL = driver.current_url + '?page={}'
    for i in range(numeric_part):
        CURL = URL.format(i+1)
        driver.get(CURL)
        time.sleep(3)
        ProPage_Link = soup.find_all('a', class_='ProductTile_productimg-link__deS5f productimg-link')
        for ProPage in ProPage_Link:
            href_value = ProPage.get('href')
            ProPage_links.append(href_value)


In [6]:
def unique(Orignal_list):
    unique_list = [item for index, item in enumerate(Orignal_list) if item not in Orignal_list[:index]]
    return unique_list

In [40]:
def get_product_info(URL):
    driver.get(URL)
    time.sleep(5)
    driver.implicitly_wait(10)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    SKU_element = soup.find_all('div', class_='OverviewSection_feature-content__Yp3Nx feature-content')
    product_name_element = soup.find('h1', class_='PDP_product-name__0qgud product-name')
    Sub_pro_element = soup.find_all('span', class_='item-label')
    regular_price_element = soup.find_all('span', class_='old-price line-through text-accents-7')
    sale_price_element = soup.find_all('span', class_='product-price--value')
    Rating_element = soup.find('div', class_='pr-snippet-rating-decimal')
    Review_Count_element = soup.find('a', class_='pr-snippet-review-count')
    for i in SKU_element:
        SKU = i.text
    product_name = product_name_element.text.strip()
    regularPrice = [element.text.strip() for element in regular_price_element] if regular_price_element else None
    salePrice = [element.text.strip() for element in sale_price_element] if sale_price_element else None
    Rating = Rating_element.text.strip() if Rating_element else 'No Reviews'
    ReviewCount= Review_Count_element.text.strip() if Review_Count_element else '0'
    df =[]
    if not Sub_pro_element:
        paths = [
        '/html/body/div[1]/div/main/div[3]/div[1]/div[2]/div/div[2]/div/div[5]/div[1]/span',
        '/html/body/div[1]/div/main/div[3]/div[1]/div[2]/div/div[2]/div/div[4]/div[1]/span'
        ]
        for path in paths:
            try:
                Sub_pro_element = driver.find_element(By.XPATH, path)
                Sub_pro = Sub_pro_element.text
                # If the element is found, break out of the loop
                break
            except NoSuchElementException as e:
                # Handle the exception, for example, print an error message
                print(f"Element not found for path {path}: {e}")
        else:
            Sub_pro = None
            print("All attempts failed. Element not found.")
            

        row = {
                'PageURL': URL,
                'SKU' : SKU,
                'ProductName': product_name,
                'Sub_Product':Sub_pro,
                'RegularPrice':regularPrice[0] if regularPrice is not None else None,
                'SalePrice':salePrice[0],
                'ReviewCount': ReviewCount,
                'OverAllRating': Rating
        }
        df.append(row)

    else:
        Sub_pro = [element.text.strip() for element in Sub_pro_element]
        for i in range(len(Sub_pro)):
            row = {
                'PageURL': URL,
                'SKU' : SKU,
                'ProductName': product_name,
                'Sub_Product':Sub_pro[i],
                'RegularPrice':regularPrice[i] if regularPrice is not None else None,
                'SalePrice':salePrice[i],
                'ReviewCount': ReviewCount,
                'OverAllRating': Rating
            }
            df.append(row)
    return df

In [2]:
def get_sitemap_urls(sitemap_url):
    response = requests.get(sitemap_url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        urls = [loc.text for loc in soup.find_all('loc')]
        return urls
    else:
        print(f"Failed to fetch sitemap. Status Code: {response.status_code}")
        return []

In [3]:
sitemap_url = 'https://www.americanmeadows.com/amisitemap.xml'
sitemap_urls = get_sitemap_urls(sitemap_url)
Products_URL= []
for i in tqdm(sitemap_urls):
    if 'https://www.americanmeadows.com/product' in i:
        Products_URL.append(i)
df = pd.DataFrame(Products_URL)
df.to_csv('SitemapLinks.csv',index=False)

C:\Users\Rahul Manocha\anaconda3\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████| 4436/4436 [00:00<00:00, 2241678.62it/s]


In [41]:
Web_URL = 'https://www.americanmeadows.com'
allowed_user_agents = [
    'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)',
    'Mozilla/5.0 (compatible; Bingbot/2.0; +http://www.bing.com/bingbot.htm)',
    
]
ProPage_links = []

In [24]:
user_agents = random.choice(allowed_user_agents)
headers = {'User-Agent': user_agents}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f"user-agent={headers['User-Agent']}")
driver = webdriver.Chrome(options=chrome_options)
driver.get(Web_URL)
driver.implicitly_wait(10)

####################################################################################################################################
for i in WildFlower():
    driver.get(i)
    time.sleep(5)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')    
    pagination = soup.find_all('a', class_='Pagination_nav-button__A1S2C')
    a = [i.find('span', class_='sr-only') for i in pagination]
    a = [i.text for i in a]
    if len(a) == 1:
        numeric_part = 1
    else:
        a = a[-2]  
        numeric_part = ''.join(char for char in a if char.isdigit())
        numeric_part = int(numeric_part)
    print(numeric_part)
    Get_ProductLinks(numeric_part)
    
####################################################################################################################################
for i in FlowerSeed():
    driver.get(i)
    time.sleep(5)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')    
    pagination = soup.find_all('a', class_='Pagination_nav-button__A1S2C')
    a = [i.find('span', class_='sr-only') for i in pagination]
    a = [i.text for i in a]
    if len(a) == 1:
        numeric_part = 1
    else:
        a = a[-2]  
        numeric_part = ''.join(char for char in a if char.isdigit())
        numeric_part = int(numeric_part)
    print(numeric_part)
    Get_ProductLinks(numeric_part)

####################################################################################################################################
wait = WebDriverWait(driver, 10)
Perennials = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[3]/nav/ul/li[3]/a")))  
Perennials.click()
time.sleep(5)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')    
pagination = soup.find_all('a', class_='Pagination_nav-button__A1S2C')
a = [i.find('span', class_='sr-only') for i in pagination]
a = [i.text for i in a]
if len(a) == 1:
    numeric_part = 1
else:
    a = a[-2]  
    numeric_part = ''.join(char for char in a if char.isdigit())
    numeric_part = int(numeric_part)
print(numeric_part)
Get_ProductLinks(numeric_part)

####################################################################################################################################
PrePlannedGardens = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[3]/nav/ul/li[4]/a")))
PrePlannedGardens.click()
time.sleep(5)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')
pagination = soup.find_all('a', class_='Pagination_nav-button__A1S2C')
a = [i.find('span', class_='sr-only') for i in pagination]
a = [i.text for i in a]
if len(a) == 1:
    numeric_part = 1
else:
    a = a[-2]  
    numeric_part = ''.join(char for char in a if char.isdigit())
    numeric_part = int(numeric_part)
print(w)
Get_ProductLinks(numeric_part)

####################################################################################################################################
flowerBulbs = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[3]/nav/ul/li[5]/a")))
flowerBulbs.click()
time.sleep(5)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')
pagination = soup.find_all('a', class_='Pagination_nav-button__A1S2C')
a = [i.find('span', class_='sr-only') for i in pagination]
a = [i.text for i in a]
if len(a) == 1:
    numeric_part = 1
else:
    a = a[-2]  
    numeric_part = ''.join(char for char in a if char.isdigit())
    numeric_part = int(numeric_part)
print(numeric_part)
Get_ProductLinks(numeric_part)

####################################################################################################################################
Lawn = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[3]/nav/ul/li[6]/a")))
Lawn.click()
time.sleep(5)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')
pagination = soup.find_all('a', class_='Pagination_nav-button__A1S2C')
a = [i.find('span', class_='sr-only') for i in pagination]
a = [i.text for i in a]
if len(a) == 1:
    numeric_part = 1
else:
    a = a[-2]  
    numeric_part = ''.join(char for char in a if char.isdigit())
    numeric_part = int(numeric_part)
print(numeric_part)
Get_ProductLinks(numeric_part)

####################################################################################################################################
Sale = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[3]/nav/ul/li[8]/a")))
Sale.click()
time.sleep(5)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')
pagination = soup.find_all('a', class_='Pagination_nav-button__A1S2C')
a = [i.find('span', class_='sr-only') for i in pagination]
a = [i.text for i in a]
if len(a) == 1:
    numeric_part = 1
else:
    a = a[-2]  
    numeric_part = ''.join(char for char in a if char.isdigit())
    numeric_part = int(numeric_part)
print(numeric_part)
Get_ProductLinks(numeric_part)

####################################################################################################################################
print(len(ProPage_links))
Unique_Links = unique(ProPage_links)
print(len(Unique_Links))
df = pd.DataFrame(Unique_Links)
df.to_csv('Links.csv',index=False)

Product_details=[]
for i in Unique_Links:
    URL = Web_URL + i
    df = get_product_info(URL)
    Product_details.extend(df)

daf = pd.DataFrame(Product_details)
daf.to_csv('AmericanMeadows.csv',index=False)
driver.quit()

3
16
16
17
16
17
16
20
11
5
9
8
8
2
2
2
3
1
2
4
4
4
2
39
9
20
5
55
5840
166
